In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

 93% 138M/149M [00:00<00:00, 198MB/s]
100% 149M/149M [00:00<00:00, 197MB/s]


In [3]:
import zipfile
zip_file_path = '/content/brain-tumor-mri-dataset.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/brain_tumor_dataset')

In [6]:
import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50


In [7]:
# Define labels and image size
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Load images and labels
x_train = []
y_train = []
x_test = []
y_test = []

for label in labels:
    trainPath = os.path.join('/content/brain_tumor_dataset/Training', label)
    for file in tqdm(os.listdir(trainPath)):
        image_path = os.path.join(trainPath, file)
        x_train.append(preprocess_image(image_path))
        y_train.append(labels.index(label))

    testPath = os.path.join('/content/brain_tumor_dataset/Testing', label)
    for file in tqdm(os.listdir(testPath)):
        image_path = os.path.join(testPath, file)
        x_test.append(preprocess_image(image_path))
        y_test.append(labels.index(label))

# Convert lists to NumPy arrays and normalize images
x_train = np.array(x_train) / 255.0
x_test = np.array(x_test) / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

# Shuffle training data
x_train, y_train = shuffle(x_train, y_train, random_state=42)

# Perform one-hot encoding on labels
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Split dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)



100%|██████████| 300/300 [00:01<00:00, 176.02it/s]


In [8]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
import numpy as np

print("TensorFlow version:", tf.__version__)
print("TensorFlow Probability version:", tfp.__version__)
print("Keras version:", keras.__version__)
print("NumPy version:", np.__version__)


TensorFlow version: 2.15.0
TensorFlow Probability version: 0.23.0
Keras version: 2.15.0
NumPy version: 1.25.2


**ResNet50**

In [9]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from time import time

# Define ResNet50-based model
def build_resnet_model(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(4, activation='sigmoid')(x)  # Adjust output units to 4
    model = models.Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False  # Freeze pre-trained layers
    return model

# Instantiate the ResNet50-based model
input_shape = (150, 150, 3)  # Assuming input shape is (150, 150, 3)
resnet_model = build_resnet_model(input_shape)

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)  # Adjust learning rate if needed
resnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time()
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)  # Add early stopping to prevent overfitting
resnet_history = resnet_model.fit(x_train, y_train,
                                  batch_size=32,
                                  epochs=10,  # Increase epochs if needed
                                  validation_data=(x_val, y_val),
                                  callbacks=[early_stopping])
end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = resnet_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {resnet_history.history["accuracy"][-1]}')  # Print final training accuracy
print(f'Validation Acc: {resnet_history.history["val_accuracy"][-1]}')  # Print final validation accuracy
print(f'Test Acc: {test_accuracy}')


94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
143/143 [==============================] - 546s 4s/step - loss: 0.5336 - accuracy: 0.4193 - val_loss: 0.4993 - val_accuracy: 0.5311
Epoch 2/10
143/143 [==============================] - 533s 4s/step - loss: 0.4792 - accuracy: 0.5572 - val_loss: 0.4728 - val_accuracy: 0.6107
Epoch 3/10
143/143 [==============================] - 534s 4s/step - loss: 0.4552 - accuracy: 0.5881 - val_loss: 0.4521 - val_accuracy: 0.5862
Epoch 4/10
143/143 [==============================] - 482s 3s/step - loss: 0.4385 - accuracy: 0.6161 - val_loss: 0.4387 - val_accuracy: 0.6133
Epoch 5/10
143/143 [==============================] - 528s 4s/step - loss: 0.4247 - accuracy: 0.6316 - val_loss: 0.4251 - val_accuracy: 0.6317
Epoch 6/10
143/143 [==============================] - 527s 4s/step - loss: 0.4141 - accuracy: 0.6380 - val_loss: 0.4131 - val_accuracy: 0.6588
Epoch 7/10
143/143 [==============================] - 484s 3s/step - loss: 0.